## Pretrained VGGNet

In [1]:
# Operations for floydhub
# !git clone https://github.com/machrisaa/tensorflow-vgg tensorflow_vgg
# !ln -s /data .

Cloning into 'tensorflow_vgg'...
remote: Counting objects: 113, done.
remote: Total 113 (delta 0), reused 0 (delta 0), pack-reused 113
Receiving objects: 100% (113/113), 55.91 KiB | 0 bytes/s, done.
Resolving deltas: 100% (61/61), done.
Checking connectivity... done.


In [1]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm

vgg_dir = 'tensorflow_vgg/'
vgg_name = 'vgg19'

# Make sure vgg exists
if not isdir(vgg_dir):
    raise Exception("VGG directory doesn't exist!")

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

vgg_param_file = '{}{}.npy'.format(vgg_dir, vgg_name)
if not isfile(vgg_param_file):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc= vgg_name + ' Parameters') as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/content.udacity-data.com/nd101/{}.npy'.format(vgg_name),
            vgg_param_file,
            pbar.hook)
else:
    print("Parameter file already exists!")

Parameter file already exists!


In [2]:
import os

import numpy as np
import tensorflow as tf

from tensorflow_vgg import vgg19
# from tensorflow_vgg import vgg16
from tensorflow_vgg import utils

In [3]:
data_dir = 'data/'
train_dir = data_dir + 'train/'

classes = [d for d in os.listdir(train_dir) if os.path.isdir(train_dir + d)]

In [4]:
from scipy.ndimage.interpolation import rotate
from skimage.transform import resize

def horizontal_flip(image):
    image = image[:, ::-1, :]
    return image

def vertical_flip(image):
    image = image[::-1, :, :]
    return image

def random_rotation(image, angle_range=(0, 180)):
    h, w, _ = image.shape
    angle = np.random.randint(*angle_range)
    image = rotate(image, angle)
    image = resize(image, (h, w))
    return image

In [ ]:
import csv

batch_size = 20
batch = []

with tf.Session() as sess:
    vgg = vgg19.Vgg19()
    # vgg = vgg16.Vgg16()
    input_ = tf.placeholder(tf.float32, [None, 224, 224, 3])
    
    with tf.name_scope("content_vgg"):
        vgg.build(input_)
    
    
    codes = None
    labels = []
    count = 0

    for d_type in ['train', 'valid']:
        for c in classes:
            image_dir = '{}{}/{}/'.format(data_dir, d_type, c) # e.g. data/train/melanoma/
            files = os.listdir(image_dir)
            for i, file in enumerate(files, 1):
                # load image and resize it to 224x224
                file_path = os.path.join(image_dir, file)
                img = utils.load_image(file_path)
                batch.append(img.reshape((1, 224, 224, 3)))
                labels.append(c)
                
                # data augumentation for all training data but nevus
                if d_type == 'train' and c != 'nevus':
                    batch.append(horizontal_flip(img).reshape((1,224,224,3)))
                    labels.append(c)
                    batch.append(vertical_flip(img).reshape((1,224,224,3)))
                    labels.append(c)
                    batch.append(random_rotation(img).reshape((1,224,224,3)))
                    labels.append(c)
                    batch.append(random_rotation(img, angle_range=(-180, 0)).reshape((1,224,224,3)))
                    labels.append(c)                  

                if (len(batch) >= batch_size) or i == len(files):
                    images = np.concatenate(batch)

                    feed_dict = {input_: images}
                    codes_batch = sess.run(vgg.relu6, feed_dict=feed_dict)

                    if codes is None:
                        codes = codes_batch
                    else:
                        codes = np.concatenate((codes, codes_batch))

                    count += len(batch)
                    batch = []
                    print('data: {}, class: {}, {} / {} images processed. data count: {}'.format(d_type, c, i, len(files), count))

        # write codes to file
        with open('{}_{}_codes'.format(vgg_name, d_type), 'w') as f:
            codes.tofile(f)
            codes = None

        # write labels to file
        with open('{}_{}_labels'.format(vgg_name, d_type), 'w') as f:
            writer = csv.writer(f, delimiter='\n')
            writer.writerow(labels)
            labels = []

        count = 0

/Users/junji/Development/udacity-deeplearning/dermatologist-ai/tensorflow_vgg/vgg19.npy
npy file loaded
build model started
build model finished: 0s


/Users/junji/miniconda3/envs/dermatologist-ai/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


data: train, class: melanoma, 4 / 374 images processed. data count: 20
data: train, class: melanoma, 8 / 374 images processed. data count: 40
data: train, class: melanoma, 12 / 374 images processed. data count: 60
data: train, class: melanoma, 16 / 374 images processed. data count: 80
data: train, class: melanoma, 20 / 374 images processed. data count: 100
data: train, class: melanoma, 24 / 374 images processed. data count: 120
data: train, class: melanoma, 28 / 374 images processed. data count: 140
data: train, class: melanoma, 32 / 374 images processed. data count: 160
data: train, class: melanoma, 36 / 374 images processed. data count: 180
data: train, class: melanoma, 40 / 374 images processed. data count: 200
data: train, class: melanoma, 44 / 374 images processed. data count: 220
data: train, class: melanoma, 48 / 374 images processed. data count: 240
data: train, class: melanoma, 52 / 374 images processed. data count: 260
data: train, class: melanoma, 56 / 374 images processed. 

## Building the Classifier

In [6]:
# read codes and labels from file
import csv

# train data
with open('{}_train_labels'.format(vgg_name)) as f:
    reader = csv.reader(f, delimiter='\n')
    train_labels = np.array([each for each in reader if len(each) > 0]).squeeze()
with open('{}_train_codes'.format(vgg_name)) as f:
    train_x = np.fromfile(f, dtype=np.float32)
    train_x = train_x.reshape((len(train_labels), -1))
    
# valid data
with open('{}_valid_labels'.format(vgg_name)) as f:
    reader = csv.reader(f, delimiter='\n')
    valid_labels = np.array([each for each in reader if len(each) > 0]).squeeze()
with open('{}_valid_codes'.format(vgg_name)) as f:
    val_x = np.fromfile(f, dtype=np.float32)
    val_x = val_x.reshape((len(valid_labels), -1))
    
# test data
with open('{}_test_labels'.format(vgg_name)) as f:
    reader = csv.reader(f, delimiter='\n')
    test_labels = np.array([each for each in reader if len(each) > 0]).squeeze()
with open('{}_test_codes'.format(vgg_name)) as f:
    test_x = np.fromfile(f, dtype=np.float32)
    test_x = test_x.reshape((len(test_labels), -1))

In [3]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
lb.fit(classes)

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [7]:
train_y = lb.transform(train_labels)
val_y = lb.transform(valid_labels)
test_y = lb.transform(test_labels)

In [11]:
print("Train shapes (x, y):", train_x.shape, train_y.shape)
print("Validation shapes (x, y):", val_x.shape, val_y.shape)
print("Test shapes (x, y):", test_x.shape, test_y.shape)

Train shapes (x, y): (10000, 4096) (10000, 3)
Validation shapes (x, y): (150, 4096) (150, 3)
Test shapes (x, y): (600, 4096) (600, 3)


In [8]:
inputs_ = tf.placeholder(tf.float32, shape=[None, train_x.shape[1]])
labels_ = tf.placeholder(tf.int64, shape=[None, train_y.shape[1]])

fc_1 = tf.contrib.layers.fully_connected(inputs_, 512)
dropout_1 = tf.contrib.layers.dropout(fc_1)
fc_2 = tf.contrib.layers.fully_connected(dropout_1, 128)
dropout_2 = tf.contrib.layers.dropout(fc_2)
    
logits = tf.contrib.layers.fully_connected(dropout_2, train_y.shape[1], activation_fn=None)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=labels_, logits=logits)
cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.RMSPropOptimizer(0.001).minimize(cost)

predicted = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(predicted, 1), tf.argmax(labels_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [9]:
def get_batches(x, y, n_batches=10):
    """ Return a generator that yields batches from arrays x and y. """
    batch_size = len(x)//n_batches
    
    for ii in range(0, n_batches*batch_size, batch_size):
        # If we're not on the last batch, grab data with size batch_size
        if ii != (n_batches-1)*batch_size:
            X, Y = x[ii: ii+batch_size], y[ii: ii+batch_size] 
        # On the last batch, grab the rest of the data
        else:
            X, Y = x[ii:], y[ii:]
        # I love generators
        yield X, Y

## Training

In [11]:
!mkdir checkpoints

epochs = 200
iteration = 0
saver = tf.train.Saver()
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    # saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    for e in range(epochs):
        for x, y in get_batches(train_x, train_y, n_batches=5):
            feed = {inputs_: x,
                    labels_: y}
            loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            print("Epoch: {}/{}".format(e+1, epochs),
                  "Iteration: {}".format(iteration),
                  "Training loss: {:.5f}".format(loss))
            iteration += 1
            
            if iteration % 5 == 0:
                feed = {inputs_: val_x, labels_: val_y}
                val_acc = sess.run(accuracy, feed_dict=feed)
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Validation Acc: {:.4f}".format(val_acc))
    saver.save(sess, "checkpoints/skin_diseases.ckpt")

mkdir: checkpoints: File exists
Epoch: 1/500 Iteration: 0 Training loss: 6.69586
Epoch: 1/500 Iteration: 1 Training loss: 6.86612
Epoch: 1/500 Iteration: 2 Training loss: 4.56314
Epoch: 1/500 Iteration: 3 Training loss: 1.46421
Epoch: 1/500 Iteration: 4 Training loss: 0.72650
Epoch: 0/500 Iteration: 5 Validation Acc: 0.4667
Epoch: 2/500 Iteration: 5 Training loss: 11.11698
Epoch: 2/500 Iteration: 6 Training loss: 5.29087
Epoch: 2/500 Iteration: 7 Training loss: 2.64127
Epoch: 2/500 Iteration: 8 Training loss: 1.10827
Epoch: 2/500 Iteration: 9 Training loss: 0.58772
Epoch: 1/500 Iteration: 10 Validation Acc: 0.4667
Epoch: 3/500 Iteration: 10 Training loss: 8.89380
Epoch: 3/500 Iteration: 11 Training loss: 4.54351
Epoch: 3/500 Iteration: 12 Training loss: 2.21916
Epoch: 3/500 Iteration: 13 Training loss: 1.02280
Epoch: 3/500 Iteration: 14 Training loss: 0.54173
Epoch: 2/500 Iteration: 15 Validation Acc: 0.4733
Epoch: 4/500 Iteration: 15 Training loss: 7.04497
Epoch: 4/500 Iteration: 16 T

Epoch: 28/500 Iteration: 135 Training loss: 6.77899
Epoch: 28/500 Iteration: 136 Training loss: 2.16448
Epoch: 28/500 Iteration: 137 Training loss: 0.83942
Epoch: 28/500 Iteration: 138 Training loss: 0.45742
Epoch: 28/500 Iteration: 139 Training loss: 0.32292
Epoch: 27/500 Iteration: 140 Validation Acc: 0.5067
Epoch: 29/500 Iteration: 140 Training loss: 4.05445
Epoch: 29/500 Iteration: 141 Training loss: 1.91020
Epoch: 29/500 Iteration: 142 Training loss: 0.79017
Epoch: 29/500 Iteration: 143 Training loss: 0.54760
Epoch: 29/500 Iteration: 144 Training loss: 0.42672
Epoch: 28/500 Iteration: 145 Validation Acc: 0.5133
Epoch: 30/500 Iteration: 145 Training loss: 2.90660
Epoch: 30/500 Iteration: 146 Training loss: 1.59857
Epoch: 30/500 Iteration: 147 Training loss: 0.77780
Epoch: 30/500 Iteration: 148 Training loss: 0.56345
Epoch: 30/500 Iteration: 149 Training loss: 0.41986
Epoch: 29/500 Iteration: 150 Validation Acc: 0.5000
Epoch: 31/500 Iteration: 150 Training loss: 2.83637
Epoch: 31/50

Epoch: 54/500 Iteration: 267 Training loss: 0.73483
Epoch: 54/500 Iteration: 268 Training loss: 0.63657
Epoch: 54/500 Iteration: 269 Training loss: 0.50221
Epoch: 53/500 Iteration: 270 Validation Acc: 0.5267
Epoch: 55/500 Iteration: 270 Training loss: 1.80072
Epoch: 55/500 Iteration: 271 Training loss: 1.25537
Epoch: 55/500 Iteration: 272 Training loss: 0.72871
Epoch: 55/500 Iteration: 273 Training loss: 0.62626
Epoch: 55/500 Iteration: 274 Training loss: 0.50198
Epoch: 54/500 Iteration: 275 Validation Acc: 0.5133
Epoch: 56/500 Iteration: 275 Training loss: 1.82496
Epoch: 56/500 Iteration: 276 Training loss: 1.28129
Epoch: 56/500 Iteration: 277 Training loss: 0.74241
Epoch: 56/500 Iteration: 278 Training loss: 0.65611
Epoch: 56/500 Iteration: 279 Training loss: 0.54979
Epoch: 55/500 Iteration: 280 Validation Acc: 0.5133
Epoch: 57/500 Iteration: 280 Training loss: 1.57133
Epoch: 57/500 Iteration: 281 Training loss: 1.19172
Epoch: 57/500 Iteration: 282 Training loss: 0.72079
Epoch: 57/50

Epoch: 80/500 Iteration: 399 Training loss: 0.44696
Epoch: 79/500 Iteration: 400 Validation Acc: 0.5133
Epoch: 81/500 Iteration: 400 Training loss: 1.60585
Epoch: 81/500 Iteration: 401 Training loss: 1.10187
Epoch: 81/500 Iteration: 402 Training loss: 0.61350
Epoch: 81/500 Iteration: 403 Training loss: 0.52065
Epoch: 81/500 Iteration: 404 Training loss: 0.43473
Epoch: 80/500 Iteration: 405 Validation Acc: 0.5533
Epoch: 82/500 Iteration: 405 Training loss: 1.54474
Epoch: 82/500 Iteration: 406 Training loss: 1.09960
Epoch: 82/500 Iteration: 407 Training loss: 0.62198
Epoch: 82/500 Iteration: 408 Training loss: 0.53301
Epoch: 82/500 Iteration: 409 Training loss: 0.43885
Epoch: 81/500 Iteration: 410 Validation Acc: 0.5267
Epoch: 83/500 Iteration: 410 Training loss: 1.54426
Epoch: 83/500 Iteration: 411 Training loss: 1.12583
Epoch: 83/500 Iteration: 412 Training loss: 0.63585
Epoch: 83/500 Iteration: 413 Training loss: 0.53037
Epoch: 83/500 Iteration: 414 Training loss: 0.42995
Epoch: 82/50

Epoch: 107/500 Iteration: 530 Training loss: 1.62446
Epoch: 107/500 Iteration: 531 Training loss: 0.98315
Epoch: 107/500 Iteration: 532 Training loss: 0.54959
Epoch: 107/500 Iteration: 533 Training loss: 0.42885
Epoch: 107/500 Iteration: 534 Training loss: 0.34383
Epoch: 106/500 Iteration: 535 Validation Acc: 0.5667
Epoch: 108/500 Iteration: 535 Training loss: 1.61238
Epoch: 108/500 Iteration: 536 Training loss: 0.96204
Epoch: 108/500 Iteration: 537 Training loss: 0.52819
Epoch: 108/500 Iteration: 538 Training loss: 0.41638
Epoch: 108/500 Iteration: 539 Training loss: 0.33370
Epoch: 107/500 Iteration: 540 Validation Acc: 0.5400
Epoch: 109/500 Iteration: 540 Training loss: 1.56894
Epoch: 109/500 Iteration: 541 Training loss: 0.96263
Epoch: 109/500 Iteration: 542 Training loss: 0.54632
Epoch: 109/500 Iteration: 543 Training loss: 0.43384
Epoch: 109/500 Iteration: 544 Training loss: 0.35183
Epoch: 108/500 Iteration: 545 Validation Acc: 0.5667
Epoch: 110/500 Iteration: 545 Training loss: 1

...

Epoch: 388/500 Iteration: 1935 Training loss: 0.61869
Epoch: 388/500 Iteration: 1936 Training loss: 0.24598
Epoch: 388/500 Iteration: 1937 Training loss: 0.14295
Epoch: 388/500 Iteration: 1938 Training loss: 0.11884
Epoch: 388/500 Iteration: 1939 Training loss: 0.08304
Epoch: 387/500 Iteration: 1940 Validation Acc: 0.6400
Epoch: 389/500 Iteration: 1940 Training loss: 0.69061
Epoch: 389/500 Iteration: 1941 Training loss: 0.24317
Epoch: 389/500 Iteration: 1942 Training loss: 0.13547
Epoch: 389/500 Iteration: 1943 Training loss: 0.11116
Epoch: 389/500 Iteration: 1944 Training loss: 0.08186
Epoch: 388/500 Iteration: 1945 Validation Acc: 0.6133
Epoch: 390/500 Iteration: 1945 Training loss: 0.70518
Epoch: 390/500 Iteration: 1946 Training loss: 0.29288
Epoch: 390/500 Iteration: 1947 Training loss: 0.18292
Epoch: 390/500 Iteration: 1948 Training loss: 0.12084
Epoch: 390/500 Iteration: 1949 Training loss: 0.08030
Epoch: 389/500 Iteration: 1950 Validation Acc: 0.6400
Epoch: 391/500 Iteration: 19

Epoch: 413/500 Iteration: 2062 Training loss: 0.12220
Epoch: 413/500 Iteration: 2063 Training loss: 0.10682
Epoch: 413/500 Iteration: 2064 Training loss: 0.08525
Epoch: 412/500 Iteration: 2065 Validation Acc: 0.6400
Epoch: 414/500 Iteration: 2065 Training loss: 0.51745
Epoch: 414/500 Iteration: 2066 Training loss: 0.20589
Epoch: 414/500 Iteration: 2067 Training loss: 0.11889
Epoch: 414/500 Iteration: 2068 Training loss: 0.10322
Epoch: 414/500 Iteration: 2069 Training loss: 0.07336
Epoch: 413/500 Iteration: 2070 Validation Acc: 0.6867
Epoch: 415/500 Iteration: 2070 Training loss: 0.56628
Epoch: 415/500 Iteration: 2071 Training loss: 0.24940
Epoch: 415/500 Iteration: 2072 Training loss: 0.14863
Epoch: 415/500 Iteration: 2073 Training loss: 0.09851
Epoch: 415/500 Iteration: 2074 Training loss: 0.06953
Epoch: 414/500 Iteration: 2075 Validation Acc: 0.6600
Epoch: 416/500 Iteration: 2075 Training loss: 0.64517
Epoch: 416/500 Iteration: 2076 Training loss: 0.32373
Epoch: 416/500 Iteration: 20

Epoch: 438/500 Iteration: 2189 Training loss: 0.09164
Epoch: 437/500 Iteration: 2190 Validation Acc: 0.6200
Epoch: 439/500 Iteration: 2190 Training loss: 0.73565
Epoch: 439/500 Iteration: 2191 Training loss: 0.29005
Epoch: 439/500 Iteration: 2192 Training loss: 0.19784
Epoch: 439/500 Iteration: 2193 Training loss: 0.11075
Epoch: 439/500 Iteration: 2194 Training loss: 0.08197
Epoch: 438/500 Iteration: 2195 Validation Acc: 0.6133
Epoch: 440/500 Iteration: 2195 Training loss: 0.74086
Epoch: 440/500 Iteration: 2196 Training loss: 0.21904
Epoch: 440/500 Iteration: 2197 Training loss: 0.15496
Epoch: 440/500 Iteration: 2198 Training loss: 0.12642
Epoch: 440/500 Iteration: 2199 Training loss: 0.10523
Epoch: 439/500 Iteration: 2200 Validation Acc: 0.6400
Epoch: 441/500 Iteration: 2200 Training loss: 0.49999
Epoch: 441/500 Iteration: 2201 Training loss: 0.30603
Epoch: 441/500 Iteration: 2202 Training loss: 0.17817
Epoch: 441/500 Iteration: 2203 Training loss: 0.11227
Epoch: 441/500 Iteration: 22

Epoch: 464/500 Iteration: 2315 Training loss: 0.44654
Epoch: 464/500 Iteration: 2316 Training loss: 0.26304
Epoch: 464/500 Iteration: 2317 Training loss: 0.13998
Epoch: 464/500 Iteration: 2318 Training loss: 0.10036
Epoch: 464/500 Iteration: 2319 Training loss: 0.07983
Epoch: 463/500 Iteration: 2320 Validation Acc: 0.6867
Epoch: 465/500 Iteration: 2320 Training loss: 0.42658
Epoch: 465/500 Iteration: 2321 Training loss: 0.18742
Epoch: 465/500 Iteration: 2322 Training loss: 0.11804
Epoch: 465/500 Iteration: 2323 Training loss: 0.08645
Epoch: 465/500 Iteration: 2324 Training loss: 0.06538
Epoch: 464/500 Iteration: 2325 Validation Acc: 0.6400
Epoch: 466/500 Iteration: 2325 Training loss: 0.48511
Epoch: 466/500 Iteration: 2326 Training loss: 0.21710
Epoch: 466/500 Iteration: 2327 Training loss: 0.13494
Epoch: 466/500 Iteration: 2328 Training loss: 0.08045
Epoch: 466/500 Iteration: 2329 Training loss: 0.07240
Epoch: 465/500 Iteration: 2330 Validation Acc: 0.6467
Epoch: 467/500 Iteration: 23

Epoch: 489/500 Iteration: 2442 Training loss: 0.11089
Epoch: 489/500 Iteration: 2443 Training loss: 0.08776
Epoch: 489/500 Iteration: 2444 Training loss: 0.06800
Epoch: 488/500 Iteration: 2445 Validation Acc: 0.6133
Epoch: 490/500 Iteration: 2445 Training loss: 0.38632
Epoch: 490/500 Iteration: 2446 Training loss: 0.19297
Epoch: 490/500 Iteration: 2447 Training loss: 0.13960
Epoch: 490/500 Iteration: 2448 Training loss: 0.07962
Epoch: 490/500 Iteration: 2449 Training loss: 0.06957
Epoch: 489/500 Iteration: 2450 Validation Acc: 0.6400
Epoch: 491/500 Iteration: 2450 Training loss: 0.42705
Epoch: 491/500 Iteration: 2451 Training loss: 0.14004
Epoch: 491/500 Iteration: 2452 Training loss: 0.10751
Epoch: 491/500 Iteration: 2453 Training loss: 0.07806
Epoch: 491/500 Iteration: 2454 Training loss: 0.05049
Epoch: 490/500 Iteration: 2455 Validation Acc: 0.6667
Epoch: 492/500 Iteration: 2455 Training loss: 0.61922
Epoch: 492/500 Iteration: 2456 Training loss: 0.22393
Epoch: 492/500 Iteration: 24

## Testing

In [13]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    feed = {inputs_: test_x,
            labels_: test_y}
    test_acc = sess.run(accuracy, feed_dict=feed)
    print("Test accuracy: {:.4f}".format(test_acc))

INFO:tensorflow:Restoring parameters from checkpoints/skin_diseases.ckpt
Test accuracy: 0.6683


## Export result to csv

In [10]:
import csv

predections_csv = open('predections.csv', 'w')
writer = csv.writer(predections_csv, lineterminator='\n')
writer.writerow(['Id', 'task_1', 'task_2'])

batch_size = 20
batch = []
labels_batch = []
files_batch = []

d_type = 'test'
    
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    vgg = vgg19.Vgg19()
    # vgg = vgg16.Vgg16()
    input_ = tf.placeholder(tf.float32, [None, 224, 224, 3])
    
    with tf.name_scope("content_vgg"):
        vgg.build(input_)
    
    for c in classes:
        image_dir = '{}{}/{}/'.format(data_dir, d_type, c) # e.g. data/train/melanoma/
        files = os.listdir(image_dir)
        for i, file in enumerate(files, 1):
            # load image and resize it to 224x224
            file_path = os.path.join(image_dir, file)
            img = utils.load_image(file_path)
            
            batch.append(img.reshape((1, 224, 224, 3)))
            labels_batch.append(c)
            files_batch.append(file_path)

            if (len(batch) >= batch_size) or i == len(files):
                images = np.concatenate(batch)

                feed_dict = {input_: images}
                codes_batch = sess.run(vgg.relu6, feed_dict=feed_dict)
                
                feed_dict = {inputs_: codes_batch, labels_: lb.transform(labels_batch)}
                predections_batch = sess.run(predicted, feed_dict=feed_dict)

                for ii in range(len(batch)):
                    predection = predections_batch[ii]
                    label = labels_batch[ii]
                    file_id = files_batch[ii]
                    
                    p_melanoma = predection[0]
                    p_seborrheic_keratosis = predection[2]
                    
                    print(file_id, p_melanoma, p_seborrheic_keratosis)
                    writer.writerow([file_id, p_melanoma, p_seborrheic_keratosis])
                    
                batch = []
                labels_batch = []
                files_batch = []
                print('data: {}, class: {}, {} / {} images processed.'.format(d_type, c, i, len(files)))
                
                
predections_csv.close()

INFO:tensorflow:Restoring parameters from checkpoints/skin_diseases.ckpt
/Users/junji/Development/udacity-deeplearning/dermatologist-ai/tensorflow_vgg/vgg19.npy
npy file loaded
build model started
build model finished: 1s


/Users/junji/miniconda3/envs/dermatologist-ai/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


data/test/melanoma/ISIC_0012258.jpg 0.00361936 0.000269527
data/test/melanoma/ISIC_0012356.jpg 0.0923471 0.000624971
data/test/melanoma/ISIC_0012369.jpg 0.000113873 1.0719e-11
data/test/melanoma/ISIC_0012395.jpg 6.12498e-06 0.999994
data/test/melanoma/ISIC_0012425.jpg 8.47347e-15 1.39262e-14
data/test/melanoma/ISIC_0012758.jpg 9.33658e-15 0.0
data/test/melanoma/ISIC_0012989.jpg 0.999901 9.90492e-05
data/test/melanoma/ISIC_0013072.jpg 0.999772 3.892e-08
data/test/melanoma/ISIC_0013073.jpg 0.930656 4.90784e-06
data/test/melanoma/ISIC_0013242.jpg 1.13367e-08 2.48399e-09
data/test/melanoma/ISIC_0013277.jpg 0.579325 0.299182
data/test/melanoma/ISIC_0013321.jpg 9.61952e-15 1.0
data/test/melanoma/ISIC_0013374.jpg 0.999034 6.12571e-13
data/test/melanoma/ISIC_0013411.jpg 1.92135e-14 8.51128e-14
data/test/melanoma/ISIC_0013414.jpg 2.21044e-23 2.51126e-30
data/test/melanoma/ISIC_0013455.jpg 7.87752e-07 2.05998e-29
data/test/melanoma/ISIC_0013457.jpg 0.790798 0.2092
data/test/melanoma/ISIC_0013459

data/test/nevus/ISIC_0013602.jpg 0.26213 6.8468e-35
data/test/nevus/ISIC_0013738.jpg 0.0 0.0
data/test/nevus/ISIC_0013794.jpg 2.48409e-31 0.0
data/test/nevus/ISIC_0013809.jpg 0.417699 2.21799e-05
data/test/nevus/ISIC_0013891.jpg 0.256449 0.000503481
data/test/nevus/ISIC_0013897.jpg 0.821308 0.00225695
data/test/nevus/ISIC_0013911.jpg 4.46607e-15 1.0
data/test/nevus/ISIC_0013966.jpg 1.56015e-05 4.12999e-13
data/test/nevus/ISIC_0013998.jpg 1.85589e-11 1.20949e-14
data/test/nevus/ISIC_0014090.jpg 2.99469e-21 7.29733e-34
data/test/nevus/ISIC_0014117.jpg 0.00276094 2.01743e-08
data/test/nevus/ISIC_0014470.jpg 1.50293e-14 0.999909
data/test/nevus/ISIC_0014675.jpg 0.0 0.0
data/test/nevus/ISIC_0014677.jpg 1.25407e-37 0.0
data/test/nevus/ISIC_0014687.jpg 0.00448983 1.00596e-06
data/test/nevus/ISIC_0014693.jpg 1.0 1.13948e-09
data/test/nevus/ISIC_0014697.jpg 0.0 0.0
data/test/nevus/ISIC_0014698.jpg 3.28989e-20 0.0
data/test/nevus/ISIC_0014720.jpg 1.36765e-05 4.39687e-15
data/test/nevus/ISIC_0014

data/test/nevus/ISIC_0015274.jpg 0.123458 1.10607e-10
data/test/nevus/ISIC_0015276.jpg 3.4326e-11 0.0
data/test/nevus/ISIC_0015279.jpg 6.07124e-22 1.5614e-35
data/test/nevus/ISIC_0015283.jpg 0.0 0.0
data/test/nevus/ISIC_0015291.jpg 1.8002e-29 0.0
data/test/nevus/ISIC_0015293.jpg 1.83167e-26 0.0
data/test/nevus/ISIC_0015298.jpg 1.01447e-22 0.0
data/test/nevus/ISIC_0015309.jpg 0.0 0.0
data/test/nevus/ISIC_0015310.jpg 3.71594e-11 1.37066e-28
data/test/nevus/ISIC_0015311.jpg 2.35275e-08 0.994787
data/test/nevus/ISIC_0015312.jpg 0.106445 5.80767e-05
data/test/nevus/ISIC_0015330.jpg 2.4146e-11 2.87893e-17
data/test/nevus/ISIC_0015331.jpg 3.81116e-29 0.0
data/test/nevus/ISIC_0015347.jpg 2.36268e-10 0.0
data/test/nevus/ISIC_0015353.jpg 0.000283289 0.9995
data/test/nevus/ISIC_0015355.jpg 1.15946e-16 4.84059e-33
data/test/nevus/ISIC_0015357.jpg 8.61728e-24 0.0
data/test/nevus/ISIC_0015360.jpg 3.20156e-09 0.0
data/test/nevus/ISIC_0015363.jpg 0.0 0.0
data/test/nevus/ISIC_0015364.jpg 1.97897e-18 1.

data/test/nevus/ISIC_0016037.jpg 0.197178 0.00189706
data/test/nevus/ISIC_0016038.jpg 1.19373e-09 1.97639e-25
data/test/nevus/ISIC_0016040.jpg 9.95452e-09 1.97141e-11
data/test/nevus/ISIC_0016041.jpg 6.22418e-15 1.0
data/test/nevus/ISIC_0016042.jpg 1.71338e-08 6.97376e-07
data/test/nevus/ISIC_0016043.jpg 1.55799e-09 5.78939e-32
data/test/nevus/ISIC_0016044.jpg 1.33503e-28 2.46856e-08
data/test/nevus/ISIC_0016045.jpg 0.007813 0.0320226
data/test/nevus/ISIC_0016046.jpg 8.87052e-05 2.65367e-20
data/test/nevus/ISIC_0016048.jpg 1.14387e-08 2.13594e-27
data/test/nevus/ISIC_0016049.jpg 0.0872385 3.55602e-12
data/test/nevus/ISIC_0016050.jpg 0.253877 2.24478e-06
data/test/nevus/ISIC_0016051.jpg 8.43852e-26 0.0
data/test/nevus/ISIC_0016052.jpg 0.371365 0.00781877
data/test/nevus/ISIC_0016053.jpg 9.40263e-11 7.28465e-25
data/test/nevus/ISIC_0016054.jpg 5.08074e-05 0.999487
data/test/nevus/ISIC_0016055.jpg 2.69768e-08 5.71192e-26
data/test/nevus/ISIC_0016056.jpg 2.18853e-09 1.30082e-16
data/test/n